## Main Ventilation Processing Script
This notebook loads IFC models, extracts areas and occupants, applies BR18 rules, calculates ventilation rates, and exports results to CSV.


### Imports
All required libraries and modules used in the main script.


In [ ]:
import math
import ifcopenshell
import csv
from typing import Dict, Tuple, Optional, List, Callable, Any
from pathlib import Path
import logging

from SpaceAreas import extract_spaces_with_area
from Occupants import count_chairs_by_space, apply_br18_occupancy, guess_annex_c_category, ANNEX_C_OCCUPANT_DENSITY
from BuildingCodes import choose_ieq_category, choose_pollution_class, ventilation_rate_method_1

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")


### Function: ask_for_ifc_paths()
Ask the user for the MEP and ARCH IFC file paths, with defaults.


In [ ]:
def ask_for_ifc_paths():
    print("Enter paths to your IFC files (press Enter to use defaults).")

    mep_default = "25-08-D-MEP.ifc"
    arch_default = "25-08-D-ARCH.ifc"

    raw_mep = input(f"MEP IFC path [{mep_default}]: ").strip()
    raw_arch = input(f"ARCH IFC path [{arch_default}]: ").strip()

    def clean(p: str, default: str) -> str:
        if not p:
            return default
        return p.strip().strip('"').strip("'")

    mep_path = clean(raw_mep, mep_default)
    arch_path = clean(raw_arch, arch_default)

    return mep_path, arch_path


### Function: load_ifc_models()
Loads the MEP and ARCH IFC files and returns the IfcOpenShell model objects.


In [ ]:
def load_ifc_models(mep_path: str | Path, arch_path: str | Path) -> Tuple[ifcopenshell.file, ifcopenshell.file, Dict[str, str]]:
    mep_path = Path(mep_path)
    arch_path = Path(arch_path)

    if not mep_path.is_file():
        raise FileNotFoundError(f"MEP IFC file not found: {mep_path}")
    if not arch_path.is_file():
        raise FileNotFoundError(f"ARCH IFC file not found: {arch_path}")
    
    logging.info(f"Opening MEP IFC model from: {mep_path}")
    MEP = ifcopenshell.open(str(mep_path))

    logging.info(f"Opening ARCH IFC model from: {arch_path}")
    ARCH = ifcopenshell.open(str(arch_path))

    logging.info(f"MEP schema: {MEP.schema}")
    logging.info(f"ARCH schema: {ARCH.schema}")

    return MEP, ARCH


### Function: simple_br18_fallback()
BR18 fallback occupancy when no chairs are found in a space.


In [ ]:
def simple_br18_fallback(space_entry):
    area = space_entry.get("area")
    if area is None or area <= 0:
        return 0

    category = guess_annex_c_category(space_entry)
    if category is None:
        if area > 200:
            logging.info(
                "Large space with no occupancy found: "
                f"Name='{space_entry.get('space_name')}', "
                f"ID={space_entry.get('space_id')}, "
                f"Area={area:.1f} m². Please check occupancy manually."
            )
        return 0

    m2_per_person = ANNEX_C_OCCUPANT_DENSITY.get(category)
    if not m2_per_person:
        return 0

    return math.ceil(area / m2_per_person)


### Main Processing Function
The entire workflow:
1. Load IFC models  
2. Ask for IEQ and pollution categories  
3. Extract space areas  
4. Count chairs per space  
5. Apply BR18 fallback  
6. Calculate ventilation  
7. Export results to CSV


#### 1. load ifc paths & models
load ifc paths from user input and open the ifc models


In [ ]:
mep_path, arch_path = ask_for_ifc_paths()
mep, arch = load_ifc_models(mep_path, arch_path)


#### 2. ask user for categories
ask the user to select ieq category and pollution class


In [ ]:
ieq_category = choose_ieq_category()
pollution_class = choose_pollution_class()


#### 3. extract space areas
extract areas from the arch ifc model and map them by space id


In [ ]:
spaces_area = extract_spaces_with_area(arch, verbose=False)
area_by_id = {s["id"]: s for s in spaces_area}


#### 4. count chairs & attach area and name
count chairs per space and attach area and name from space extraction


In [ ]:
spaces_chairs = count_chairs_by_space(arch, verbose=False)

for space in spaces_chairs:
    space_id = space["space_id"]
    area_info = area_by_id.get(space_id)

    if area_info is not None:
        space["area"] = area_info["area"]
        if area_info.get("name"):
            space["space_name"] = area_info["name"]
    else:
        space["area"] = None


#### 5. apply br18 fallback
apply br18 fallback occupancy for spaces without chairs


In [ ]:
spaces_with_occupants = apply_br18_occupancy(
    spaces_chairs,
    br18_fallback=simple_br18_fallback,
    verbose=False,
)


#### 6. build csv header rows
prepare csv metadata rows and column headers


In [ ]:
rows = []

rows.append([
    "Building Pollution Category",
    pollution_class,
    "", "", "", "", "",
])
rows.append([
    "Indoor Air Quality Category",
    ieq_category,
    "", "", "", "", "",
])
rows.append([
    "Numbering",
    "Space ID",
    "Space Name",
    "Area [m²]",
    "Occupancy [persons]",
    "Source of Occupancy",
    "Calculated Vent Rates [L/s]",
])


#### 7. compute ventilation, append rows & export csv
calculate ventilation rates for each space, append rows, and write the csv file


In [ ]:
numbering = 1
for space in spaces_with_occupants:
    space_id = space.get("space_id")
    name = space.get("space_name") or ""
    area = space.get("area")
    occupants = space.get("final_occupants", 0)
    source = space.get("occupant_source", "")

    area_value = float(area) if area is not None else 0.0

    q_tot = ventilation_rate_method_1(
        occupants=occupants,
        area=area_value,
        ieq_category=ieq_category,
        pollution_class=pollution_class,
    )

    rows.append([
        numbering,
        space_id,
        name,
        area if area is not None else "",
        occupants,
        source,
        round(q_tot, 2),
    ])

    numbering += 1

output_path = Path("ventilation_results.csv")
with output_path.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(rows)

print(f"CSV written to: {output_path.resolve()}")


### Run the Script
Runs the `main()` function.


In [ ]:
if __name__ == "__main__":
    main()
